In [5]:
!pip install datasets
!pip install nltk
!pip install gensim
!pip install konlpy

     |████████████████████████████████| 542 kB 8.6 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 118 kB 17.7 MB/s 
     |████████████████████████████████| 243 kB 18.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 1.1 MB/s 
     |████████████████████████████████| 448 kB 63.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os

os.chdir('/content/drive/MyDrive/nlp/mulcam/paper/convolutional-neural-networks-for-sentence-classification/src')

In [ ]:
#load data
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("nsmc")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

import re

docs = dataset['train']['document'] + dataset['test']['document']
label = dataset['train']['label'] + dataset['test']['label']

processed_docs = [re.sub("[\s]+", " ", re.sub("[^°¡-ÆRa-zA-Z0-9]", " ", doc)) for doc in docs]

from konlpy.tag import Okt
import pickle

okt = Okt()

res = []

import os

if not os.path.isfile('senspos.pickle'):
  for doc in docs:
    tokenlist = okt.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
        temp.append(w[0])
    res.append(temp)

  with open("senspos.pickle", 'wb') as f:
    pickle.dump(res, f)
  
else:
  with open("senspos.pickle", 'rb') as f:
    res = pickle.load(f)



# padding
maxlen = max(len(x) for x in res)
padded_sens = []
for i in range(len(res)):
  sen = res[i]
  temp = sen + [" <PAD/>"] * (maxlen - len(sen))
  padded_sens.append(temp)


# vocab to index
import nltk
tokens = [t for d in padded_sens for t in d]
text = nltk.Text(tokens, name='NSMC')
word_count = text.vocab()
vocabulary_inv = [x[0] for x in word_count.most_common()]
vocabulary = {x:i for i, x in enumerate(vocabulary_inv)}


# padded sens to index sens
import numpy as np

x = np.array([[vocabulary[w] for w in sentence] for sentence in padded_sens])
y = np.array(label)

# word2vec
from gensim.models import word2vec
import numpy as np

w2v = word2vec.Word2Vec.load("../data/w2v/ko.bin")

w2v = {k: w2v[w] if w in w2v else np.random.uniform(-0.25, 0.25, w2v.vector_size) for w, k in vocabulary.items()}


# args setting
embedding_dim = 200
filter_sizes = (3, 4, 5)
num_filters = 100
dropout = 0.5
hidden_dims = 100

batch_size = 50
num_epochs = 10
min_word_count = 1
context = 10

# make dataset to use at training

X_train = x[:len(train_df)]
y_train = y[:len(train_df)]
X_test = x[len(train_df):]
y_test = y[len(train_df):]

del x, y, tokens, text, word_count, padded_sens, res, docs, label, train_df, test_df, dataset, vocabulary

# setting model and train
from tensorflow import keras

inp = keras.layers.Input(shape=(X_test.shape[1], ))

m = keras.layers.Embedding(len(vocabulary_inv), embedding_dim, input_length=X_test.shape[1], name="embedding")(inp) # non-static-cnn
m = keras.layers.Dropout(dropout)(m)

# build conv block
conv_blocks = []
for s in filter_sizes:
  conv = keras.layers.Conv1D(filters=num_filters,
                         kernel_size=s,
                         padding="valid",
                         activation="relu",
                         strides=1)(m)
  conv = keras.layers.MaxPooling1D(pool_size=2)(conv)
  conv = keras.layers.Flatten()(conv)
  conv_blocks.append(conv)
m = keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

m = keras.layers.Dropout(dropout)(m)
m = keras.layers.Dense(hidden_dims, activation="relu")(m)
out = keras.layers.Dense(1, activation="sigmoid")(m)

model = keras.Model(inp, out)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# add embedding layer for non-static-cnn
w = np.array([v for v in w2v.values()])
embedding_layer = model.get_layer("embedding")
embedding_layer.set_weights([w])

model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(X_test, y_test), verbose=2)


Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Epoch 1/10
3000/3000 - 659s - loss: 0.5410 - accuracy: 0.7295 - val_loss: 0.4066 - val_accuracy: 0.8255
Epoch 2/10
